<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/pre-processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제일먼저 구글드라이버에 nmt-fold에 code1과 data1이 설치되도록 자신의 구글드라이버에 제공되는 zip 파일을 압축 해제해서 upload 하세요. 그리고 좌측 mount 버튼을 누르세요. 그러면 구글 드라이브 마운트 위치 "/content/drive/MyDrive/"에 마운트가 되어서 /content/drive/MyDrive/nmt_fold/data1/ 형태로 접근이 됩니다.

In [ ]:
!cat /content/drive/MyDrive/nmt_fold/data1/org.*.txt > /content/drive/MyDrive/nmt_fold/data1/combined.tsv

인코딩을 UTF_16LE에서 UTF-8로 변경

In [ ]:
!iconv -f UTF-16LE -t UTF-8 /content/drive/MyDrive/nmt_fold/data1/combined.tsv > /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv
!mv /content/drive/MyDrive/nmt_fold/data1/combined_utf8.tsv /content/drive/MyDrive/nmt_fold/data1/combined.tsv

In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined*.tsv
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined*.tsv

In [ ]:
!ls -l /content/drive/MyDrive/nmt_fold/data1/

160만 문장을 섞는다.

In [ ]:
!shuf /content/drive/MyDrive/nmt_fold/data1/combined.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv

120만개 train,20만개 valid, 202418개 test 나누기

In [ ]:
!head -n 1200000 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv >/content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv
!tail -n 402418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv | head -n 200000 > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv
!tail -n 202418 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv

앞의 column은 한글, 뒤의 column은 영어, 구분해서 파일로 저장

In [ ]:
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.en
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en
!cut -f1 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko
!cut -f2 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tsv > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.en

In [ ]:
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*
!head -n 3 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.ko
!head -n 3 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.en

mecab을 설치한다.

In [ ]:
#!chmod +x /content/drive/MyDrive/nmt_fold/code1/install_mecab_linux.sh
#!bash /content/drive/MyDrive/nmt_fold/code1/install_mecab_linux.sh
# 위 두줄로 설치하다 오류나서 아래와 같이 설치함.

# 1) 필수 도구
!apt-get -y update
!apt-get -y install curl

# 2) konlpy 공식 스크립트로 mecab + mecab-ko-dic 설치
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# 3) mecabrc의 dicdir 설정(로그에 나온 경로로 맞추기)
!sed -i 's|^dicdir =.*|dicdir = /usr/local/lib/mecab/dic/mecab-ko-dic|' /usr/local/etc/mecabrc

# 4) 파이썬 라이브러리
!pip install konlpy

mecab 설치 확인하기

In [ ]:
!mecab -D | head -n 20

In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko | mecab -O wakati

Mecab으로 tokenization하고 post_tokenizer를 가지고 공백에 특수문자를 대체한다. 이는 한글에 대한 tokenization 방법이다.

In [ ]:
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.ko
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko | mecab -O wakati -b 99999  | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.ko > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.ko


In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.ko
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.ko


한글 tokenization된것 원래대로 5문장에 대해서 되돌려본 코드

In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko | python /content/drive/MyDrive/nmt_fold/code1/detokenizer.py

 에러가 나서 아래에 mosestokenizer를 따로 설치함.

In [ ]:
#!pip install mosestokenizer
!pip install -q sacremoses

영어 tokenizer에서 "깨지는 문제에 때문에 tokenizer2.py를 사용해 주었음.

In [ ]:
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.en | python /content/drive/MyDrive/nmt_fold/code1/tokenizer2.py | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.en > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.en
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en | python /content/drive/MyDrive/nmt_fold/code1/tokenizer2.py | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.en> /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.en
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.en | python /content/drive/MyDrive/nmt_fold/code1/tokenizer2.py | python /content/drive/MyDrive/nmt_fold/code1/post_tokenize.py /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.en > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.en

In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko
#!tail -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.en
#!tail -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.en
#!ls /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.tok.en

In [ ]:
!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.en | python /content/drive/MyDrive/nmt_fold/code1/detokenizer.py

In [ ]:
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.tok.ko

BPE를 적용시키기 위해서 학습하는 과정(아래 코드는 영어, 한글 순대로 배열) => 학습 방식을 김기현 버전에서 변화시킴. _를 별로의 문자로 처리해주는 방향. 왜냐하면 "_있" "있" 이 제일 많은 결합으로 나오는 오류가 원래 버전에 있었음 이는 _를 처리하는 방식에서 나오는 오류임

In [ ]:
!python /content/drive/MyDrive/nmt_fold/code1/learn_bpe2.py --input /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.en --output /content/drive/MyDrive/nmt_fold/data1/bpe.en.model --symbols 50000 --verbose
#!python /content/drive/MyDrive/nmt_fold/code1/learn_bpe2.py --input /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko --output /content/drive/MyDrive/nmt_fold/data1/bpe.ko.model --symbols 30000 --verbose

In [ ]:
!head -30 /content/drive/MyDrive/nmt_fold/data1/bpe.en.model
#!wc -l /content/drive/MyDrive/nmt_fold/data1/bpe.en.model

학습한 BPE를 적용시켜 subword segmentation 하는 과정

In [ ]:
#!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.ko | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.ko.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.bpe.ko
#!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.ko | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.ko.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.bpe.ko
#!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.ko | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.ko.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.bpe.ko
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.en | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.en.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.train.tok.bpe.en
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.en | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.en.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.test.tok.bpe.en
!cat /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.en | python /content/drive/MyDrive/nmt_fold/code1/apply_bpe2.py -c /content/drive/MyDrive/nmt_fold/data1/bpe.en.model > /content/drive/MyDrive/nmt_fold/data1/combined.shuf.valid.tok.bpe.en

In [ ]:
#!head -n 5 /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.tok.bpe.*
!wc -l /content/drive/MyDrive/nmt_fold/data1/combined.shuf.*.tok.bpe.*